In [1]:
#!/usr/bin/env python3
from std_srvs.srv import Empty, Trigger, TriggerRequest
import smach
from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import PoseStamped, Point , Quaternion, Twist, WrenchStamped
from actionlib_msgs.msg import GoalStatus
import moveit_commander
import moveit_msgs.msg
import tf2_ros as tf2
from tf2_sensor_msgs.tf2_sensor_msgs import do_transform_cloud
import controller_manager_msgs.srv
import rospy
import trajectory_msgs.msg
#from object_classification.srv import *
from sensor_msgs.msg import Image as ImageMsg
from cv_bridge import CvBridge, CvBridgeError

import cv2 as cv
import numpy as np

from utils_notebooks import *
from utils_takeshi import *
from grasp_utils import *

In [2]:
global head, whole_body, arm, tfbuff, lis, broad, tf_static_broad, scene, robot
global rgbd, hand_cam, gripper, clear_octo_client, service_client, AR_starter, AR_stopper

moveit_commander.roscpp_initialize(sys.argv)
rospy.init_node('Pruebas_de_graspeo')
head = moveit_commander.MoveGroupCommander('head')
whole_body = moveit_commander.MoveGroupCommander('whole_body_light')
arm =  moveit_commander.MoveGroupCommander('arm')
    
tfbuff = tf2.Buffer()
lis = tf2.TransformListener(tfbuff)
broad = tf2.TransformBroadcaster()
tf_static_broad = tf2.StaticTransformBroadcaster()
whole_body.set_workspace([-6.0, -6.0, 6.0, 6.0]) 
    
scene = moveit_commander.PlanningSceneInterface()
robot = moveit_commander.RobotCommander()
rgbd = RGBD()
hand_cam = HAND_RGB()
gripper = GRIPPER()

clear_octo_client = rospy.ServiceProxy('/clear_octomap', Empty)
service_client = rospy.ServiceProxy('/segment_2_tf', Trigger)
AR_starter = rospy.ServiceProxy('/marker/start_recognition',Empty)
AR_stopper = rospy.ServiceProxy('/marker/stop_recognition',Empty)

[ WARN] [1664933691.244007353, 138.462000000]: Link hand_l_finger_vacuum_frame has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1664933691.249546045, 138.462000000]: Link head_l_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1664933691.249758690, 138.462000000]: Link head_r_stereo_camera_link has visual geometry but no collision geometry. Collision geometry will be left empty. Fix your URDF file by explicitly specifying collision geometry.
[ WARN] [1664933691.251375841, 138.462000000]: Group state 'neutral' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WARN] [1664933691.251544193, 138.462000000]: Group state 'go' doesn't specify all group joints in group 'arm'. wrist_ft_sensor_frame_joint is missing.
[ WA

In [3]:
def tf2_obj_2_arr(transf):
    trans = []
    trans.append(transf.transform.translation.x)
    trans.append(transf.transform.translation.y)
    trans.append(transf.transform.translation.z)
    
    rot = []
    rot.append(transf.transform.rotation.x)
    rot.append(transf.transform.rotation.y)
    rot.append(transf.transform.rotation.z)
    rot.append(transf.transform.rotation.w)
    
    return [trans, rot]

[ WARN] [1664933717.784155575, 143.547000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 143.610000 according to authority /pose_integrator
[ WARN] [1664933717.784629404, 143.547000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 143.610000 according to authority unknown_publisher


In [4]:
base = 

array([[[ 33,  33,  33],
        [ 34,  34,  34],
        [ 35,  35,  35],
        ...,
        [181, 181, 181],
        [176, 176, 176],
        [179, 179, 179]],

       [[ 32,  32,  32],
        [ 32,  32,  32],
        [ 37,  37,  37],
        ...,
        [176, 176, 176],
        [178, 178, 178],
        [177, 177, 177]],

       [[ 34,  34,  34],
        [ 33,  33,  33],
        [ 34,  34,  34],
        ...,
        [178, 178, 178],
        [180, 180, 180],
        [176, 176, 176]],

       ...,

       [[ 66,  66,  66],
        [ 67,  67,  67],
        [ 67,  67,  67],
        ...,
        [175, 175, 175],
        [179, 179, 179],
        [179, 179, 179]],

       [[ 65,  65,  65],
        [ 66,  66,  66],
        [ 65,  65,  65],
        ...,
        [178, 178, 178],
        [178, 178, 178],
        [177, 177, 177]],

       [[ 66,  66,  66],
        [ 66,  66,  66],
        [ 66,  66,  66],
        ...,
        [177, 177, 177],
        [177, 177, 177],
        [175, 175, 175]]

[ WARN] [1664933744.316953402, 148.638000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 148.680000 according to authority unknown_publisher
[ WARN] [1664933744.322169450, 148.638000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 148.680000 according to authority /pose_integrator
[ WARN] [1664933764.890214438, 152.217000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 152.244000 according to authority unknown_publisher
[ WARN] [1664933764.892506714, 152.217000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 152.244000 according to authority /pose_integrator
[ WARN] [1664933765.607142032, 152.331000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 152.373000 according to authority /pose_integrator
[ WARN] [1664933765.609092082, 152.331000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame 

[ WARN] [1664933791.060875312, 156.618000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 156.642000 according to authority unknown_publisher
[ WARN] [1664933791.065058073, 156.618000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 156.642000 according to authority /pose_integrator
[ WARN] [1664933793.129803959, 156.951000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 157.032000 according to authority /pose_integrator
[ WARN] [1664933793.134381552, 156.951000000]: TF_REPEATED_DATA ignoring data with redundant timestamp for frame odom at time 157.032000 according to authority unknown_publisher
